#DATA.PY

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#importing libraries
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
#train_test_split is to spliting dataset in to train, test, vaid
from sklearn.model_selection import train_test_split

# 80 for training, 10 for validation, 10 for testing
def load_data(path, split=0.1):

  #loading image and mask from cvc-612
  #glob is for geeting all the conent available in the file
  #sorting used for arrange image with its repective mask 
  # path="/content/drive/MyDrive/pp1/PROJECT/PROJECT/CVC-612"
    images = sorted(glob(os.path.join(path, "/content/drive/MyDrive/images/*")))
    masks = sorted(glob(os.path.join(path, "/content/drive/MyDrive/masks/*")))

#calculating the size of the iamges
    total_size = len(images)
#spliting images in to validsize and testsize    
    valid_size = int(split * total_size)
    test_size = int(split * total_size)
    #print(total_size, valid_size, test_size)

#spliting train data into train and validation
    train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=42)
#spliting training data into train and test
    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)
    
    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)
#reading the image and converting it to numpy array
def read_image(path):
    path = path.decode()#beacuse the path is in binary format
    x = cv2.imread(path, cv2.IMREAD_COLOR)#geting images into RGB format
    x = cv2.resize(x, (224, 224))
    x = x/255.0 #normalizing the array so that its value will come between 0 and 1, it will help to reduce time.
    # (256,256,3)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)#geting masks in grayscale
    x = cv2.resize(x, (224, 224))
    x = x/255.0 
    #(256,256)
    x = np.expand_dims(x, axis=-1)#expanding dimension of the numpy array.
    #(256,256,1)
    return x  

def tf_parse(x, y):#The tf_parse function parses a single image and mask path.
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

#read_image and read_mask returning numpy array so we have to convert it in to tensorflow usable format
    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([224, 224, 3])
    y.set_shape([224, 224, 1])
    return x, y
#tf.data pipeline which takes a list of images, masks paths and the batch size.
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)#making batch of 8 images
    dataset = dataset.repeat()
    return  dataset

if __name__ =="__main__":
    path="/content/drive/MyDrive"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y)= load_data(path) 
    
    #for testing DATA.py
    ds = tf_dataset(test_x,test_y)
    for x,y in ds:
      print(x.shape, y.shape)
      break
         

(8, 224, 224, 3) (8, 224, 224, 1)


#MODEL.PY

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

#x is the input num_filters is the number of filter we want to use in cn layer
def conv_block(x, num_filters):
    x = Conv2D(num_filters, (3, 3), padding="same")(x)#using convolutional 
    x = BatchNormalization()(x)#batch normalization improves the performences
    x = Activation("relu")(x)

    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def build_model():
    size = 224 #input size
    num_filters = [16, 32, 48, 64] #filters can go on like 128, 256
    inputs = Input((size, size, 3))#setting the input layer (256, 256, 3)

    skip_x = []#creating a list to store x's information
    x = inputs
    ## Encoder
    for f in num_filters:
        x = conv_block(x, f)
        skip_x.append(x)#added append skip connection to the list
        x = MaxPool2D((2, 2))(x)#performing 2D max pooling

    ## Bridge
    x = conv_block(x, num_filters[-1])#using last filter to join the decoder part with encoder

    num_filters.reverse()#reversing the filters
    skip_x.reverse()#reversing the skip connection
    ## Decoder
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2))(x)#upsampling used to copies information from one pixel to another
        xs = skip_x[i]#extracting information from the list
        x = Concatenate()([x, xs])
        x = conv_block(x, f)
        #print(type(x))
    ## Output
    x = Conv2D(1, (1, 1), padding="same")(x)
    x = Activation("sigmoid")(x)#using sigmoid activation to get out in 0 and 1

    return Model(inputs, x)


if __name__ == "__main__":
    model = build_model()
    model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 224, 224, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             

#TRAIN.PY

In [ ]:
from tensorflow.python.ops.gen_dataset_ops import shuffle_dataset
from numpy.random.mtrand import shuffle
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from tensorflow.keras.metrics import Recall, Precision
#from data import load_data, tf_dataset
#from model import build_model

def iou(y_true, y_pred):#intersection over union
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)#adding 1e-15 because union's value can be very small
        x = x.astype(np.float32)#converting to float32
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

if __name__ == "__main__":

  ##seeding 
    np.random.seed(42)
    tf.random.set_seed(42)
    ## Dataset
    path = "/content/drive/MyDrive/"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)

    ## Hyperparameters
    batch = 8
    lr = 1e-4
    epochs = 50

    train_dataset = tf_dataset(train_x, train_y, batch=batch)#training pipe line
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)#validating pipeline
   
   #building the u net model
    model = build_model()

    opt = tf.keras.optimizers.Adam(lr)
    metrics = ["acc", Recall(), Precision(), iou]
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=metrics)#as we want to get loss in binary

    callbacks = [
        ModelCheckpoint("/content/drive/MyDrive/files/model.h5"),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3),#if the validation loss does not reduce after 3 patience gactor will increase by 0.1
        CSVLogger("/content/drive/MyDrive/files/data.csv"),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)#to avoid overfitting
    ]

    train_steps = len(train_x)//batch#defining number of batches in train_x
    valid_steps = len(valid_x)//batch#defining number of batches in valid_x

    if len(train_x) % batch != 0:#if batch=0 implementing train_steps by 1
        train_steps += 1
    if len(valid_x) % batch != 0:#if batch=0 implementing valid_steps by 1
        valid_steps += 1

    model.fit(
        train_dataset,
        validation_data=valid_dataset,
        epochs=epochs,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
        #shuffle_dataset = False
        )

Epoch 1/50
40/40 [==============================] - 198s 5s/step - loss: 0.6045 - acc: 0.6814 - recall_42: 0.6497 - precision_42: 0.2960 - iou: 0.1703 - val_loss: 0.6401 - val_acc: 0.8293 - val_recall_42: 0.0434 - val_precision_42: 0.3957 - val_iou: 0.1361 - lr: 1.0000e-04
Epoch 2/50
40/40 [==============================] - 190s 5s/step - loss: 0.5109 - acc: 0.7615 - recall_42: 0.6748 - precision_42: 0.3834 - iou: 0.1953 - val_loss: 0.5562 - val_acc: 0.8329 - val_recall_42: 2.0006e-04 - val_precision_42: 0.1134 - val_iou: 0.1282 - lr: 1.0000e-04
Epoch 3/50
40/40 [==============================] - 188s 5s/step - loss: 0.4511 - acc: 0.8134 - recall_42: 0.6614 - precision_42: 0.4657 - iou: 0.2146 - val_loss: 0.5065 - val_acc: 0.8331 - val_recall_42: 0.0000e+00 - val_precision_42: 0.0000e+00 - val_iou: 0.1197 - lr: 1.0000e-04
Epoch 4/50
40/40 [==============================] - 191s 5s/step - loss: 0.4057 - acc: 0.8532 - recall_42: 0.6508 - precision_42: 0.5610 - iou: 0.2373 - val_loss: 0.4

#PREDICT.PY

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
#from data import load_data, tf_dataset
#from train import iou

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (224, 224))
    x = x/255.0
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (224, 224))
    x = np.expand_dims(x, axis=-1)
    return x
#for joining of image and predicted mask
def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask


if __name__ == "__main__":
    ## Dataset
    path = "/content/drive/MyDrive/"
    batch_size = 16
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)
    print(len(train_x), len(valid_x), len(test_x))
    test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

    test_steps = len(test_x)//batch
    if len(test_x) % batch != 0:
        test_steps += 1
#loading the u net model
    with CustomObjectScope({'iou': iou}):
        model = tf.keras.models.load_model("/content/drive/MyDrive/files/model.h5")
#evaluateing the model
       # inputs=tf.Tensor(shape=(8,), dtype=str)
        #training=False
        #mask=None
        #model.evaluate(test_dataset , steps=test_steps)
    for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
        x = read_image(x)
        y = read_mask(y)
        y_pred = model.predict(np.expand_dims(x, axis=0))
        y_pred= y_pred[0]>0.5#if value of any pixel is gratern than 0.5 then it will work else it will be 0
        h, w, _ = x.shape
        white_line = np.ones((h, 10, 3)) * 255.0

        opt = tf.keras.optimizers.Adam(lr)
        metrics = ["acc", tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), iou]
        model.compile(loss="binary_crossentropy", optimizer=opt, metrics=metrics)

        all_images = [
            x*255.0 , white_line,
            mask_parse(y), white_line,
            mask_parse(y_pred)*255.0
        ]
        image = np.concatenate(all_images, axis=1)
        cv2.imwrite(f"/content/drive/MyDrive/results/{i}.png", image)

   

320 40 40


  0%|          | 0/40 [00:00<?, ?it/s]

1/1 [==============================] - 1s 606ms/step


  2%|▎         | 1/40 [00:00<00:34,  1.14it/s]

1/1 [==============================] - 1s 844ms/step


...

 95%|█████████▌| 38/40 [00:28<00:01,  1.62it/s]

1/1 [==============================] - 1s 542ms/step


 98%|█████████▊| 39/40 [00:28<00:00,  1.59it/s]

1/1 [==============================] - 1s 858ms/step


100%|██████████| 40/40 [00:29<00:00,  1.34it/s]


#GRADCAM.PY

In [ ]:
!pip install tensorflow numpy matplotlib opencv-python-headless

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.models import Model

import tensorflow as tf

import numpy as np
import imutils
import cv2

import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
with CustomObjectScope({'iou': iou}):
        model = VGG16(tf.keras.models.load_model("/content/drive/MyDrive/files/model.h5"))
# load the original image from disk (in OpenCV format) and then
# resize the image to its target dimensions
orig = cv2.imread("/content/drive/MyDrive/images/0004a718-546c-41c2-9c69-c4685093a039.jpg")
resized = cv2.resize(orig, (224, 224))
# load the input image from disk (in Keras/TensorFlow format) and
# preprocess it
image = load_img("/content/drive/MyDrive/images/0004a718-546c-41c2-9c69-c4685093a039.jpg", target_size=(224, 224))
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
image = imagenet_utils.preprocess_input(image)
# use the network to make predictions on the input imag and find
# the class label index with the largest corresponding probability
preds = model.predict(image)
i = np.argmax(preds[0])
# decode the ImageNet predictions to obtain the human-readable label
decoded = imagenet_utils.decode_predictions(preds)
(imagenetID, label, prob) = decoded[0][0]
label = "{}: {:.2f}%".format(label, prob * 100)
#print("[INFO] {}".format(label))
class GradCAM:
    def __init__(self, model, classIdx, layerName=None):
        # store the model, the class index used to measure the class
        # activation map, and the layer to be used when visualizing
        # the class activation map
        self.model = model
        self.classIdx = classIdx
        self.layerName = layerName

        # if the layer name is None, attempt to automatically find
        # the target output layer
        if self.layerName is None:
            self.layerName = self.find_target_layer()

    def find_target_layer(self):
        # attempt to find the final convolutional layer in the network
        # by looping over the layers of the network in reverse order
        for layer in reversed(self.model.layers):
            # check to see if the layer has a 4D output
            if len(layer.output.shape) == 4:
                return layer.name

        # otherwise, we could not find a 4D layer so the GradCAM
        # algorithm cannot be applied
        raise ValueError("Could not find 4D layer. Cannot apply GradCAM.")

    def compute_heatmap(self, image, eps=1e-8):
        # construct our gradient model by supplying (1) the inputs
        # to our pre-trained model, (2) the output of the (presumably)
        # final 4D layer in the network, and (3) the output of the
        # softmax activations from the model
        gradModel = Model(inputs=[self.model.inputs], outputs= [self.model.get_layer(self.layerName).output, self.model.output])

        # record operations for automatic differentiation
        with tf.GradientTape() as tape:
            # cast the image tensor to a float-32 data type, pass the
            # image through the gradient model, and grab the loss
            # associated with the specific class index
            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = gradModel(inputs)
            loss = predictions[:, self.classIdx]

        # use automatic differentiation to compute the gradients
        grads = tape.gradient(loss, convOutputs)

        # compute the guided gradients
        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads

        # the convolution and guided gradients have a batch dimension
        # (which we don't need) so let's grab the volume itself and
        # discard the batch
        convOutputs = convOutputs[0]
        guidedGrads = guidedGrads[0]

        # compute the average of the gradient values, and using them
        # as weights, compute the ponderation of the filters with
        # respect to the weights
        weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)

        # grab the spatial dimensions of the input image and resize
        # the output class activation map to match the input image
        # dimensions
        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))

        # normalize the heatmap such that all values lie in the range
        # [0, 1], scale the resulting values to the range [0, 255],
        # and then convert to an unsigned 8-bit integer
        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        heatmap = (heatmap * 255).astype("uint8")

        # return the resulting heatmap to the calling function
        return heatmap

    def overlay_heatmap(self, heatmap, image, alpha=0.5,
        colormap=cv2.COLORMAP_JET):
        # apply the supplied color map to the heatmap and then
        # overlay the heatmap on the input image
        heatmap = cv2.applyColorMap(heatmap, colormap)
        output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)

        # return a 2-tuple of the color mapped heatmap and the output,
        # overlaid image
        return (heatmap, output)
# initialize our gradient class activation map and build the heatmap
cam = GradCAM(model, i)
heatmap = cam.compute_heatmap(image)

# resize the resulting heatmap to the original input image dimensions
# and then overlay heatmap on top of the image
heatmap = cv2.resize(heatmap, (orig.shape[1], orig.shape[0]))
(heatmap, output) = cam.overlay_heatmap(heatmap, orig, alpha=0.5)

# draw the predicted label on the output image
cv2.rectangle(output, (0, 0), (340, 40), (0, 0, 0), -1)
cv2.putText(output, label, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

# display the original image and resulting heatmap and output image
# to our screen
#output = np.concatenate([orig, heatmap, output], axis=1)
output = np.vstack([orig, heatmap, output])
output = imutils.resize(output, height=700)
cv2.imwrite(f"/content/drive/MyDrive/gradcam/{i}.png", output)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: ignored